In [10]:
import cv2
import matplotlib.pyplot as plt 
from IPython import display
import numpy as np

In [11]:
net = cv2.dnn.readNet('./net/yolov3.cfg', './net/yolov3.weights')

layer_names = net.getLayerNames()
out_layers_indexes = net.getUnconnectedOutLayers()
out_layers = [layer_names[index[0]-1] for index in out_layers_indexes]

with open('coco.names.txt') as f:
    classes = f.read().split('\n')

In [12]:
def draw_object(img, index, score, box):
    x,y,w,h = box
    # positioning the rectangle
    start = (x, y) # x, y first point of rectangle
    end = (x + w, y + h) # x, y second point
    color = (0, 255, 0)
    width = 2
    img = cv2.rectangle(img, start, end, color, width)
    # positioning the text
    start = (x, y-10)
    font_size = 5
    font = cv2.FONT_HERSHEY_SIMPLEX
    color = (0, 255, 0)
    width = 2
    text = classes[index]

    img = cv2.putText(img, text, start, font, font_size, color, width, cv2.LINE_AA)

    return img

In [ ]:
def apply_yolo(img):
    height, width, depth = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255, (608, 608), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)

    class_scores = []
    class_indexes = []
    boxes = []

    for out in outs:
        for obj in out:
            scores = obj[5:]
            class_index = np.argmax(scores)
            class_score = scores[class_index]
            if class_score > 0:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                obj_width = int(obj[2] * width)
                obj_height = int(obj[3] * height)

                x = center_x - obj_width // 2
                y = center_y - obj_height // 2

                box = [x, y, obj_width, obj_height]
                boxes.append(box)
                class_indexes.append(class_index)
                class_scores.append(float(class_score))

    chosen_boxes = cv2.dnn.NMSBoxes(boxes, class_scores, 0, 0.4)

    for box_index in chosen_boxes:
        box_index = box_index[0]
        img = draw_object(img, class_indexes[box_index], class_scores[box_index], boxes[box_index])

    return img

In [20]:
size = (3840, 2160) # must be the same as input video size
codec = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', codec, 20, size)

cap = cv2.VideoCapture('./src/video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = apply_yolo(frame)
    out.write(frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (2500//2, 1440//2))

    plt.figure(figsize=(20,15))
    plt.imshow(frame)
    display.clear_output(wait=True)
    display.display(plt.gcf())

cap.release()
out.release()